In [1]:
import numpy as np
import pandas as pd 
import pandas_profiling as pp
import plotly.express as px 
import sqlite3
import random

pd.options.display.max_columns = None

## Import des datasets 

In [2]:
df_aeroports = pd.read_parquet("../data/parquet_format/train_data/aeroports.gzip")
df_compagnies = pd.read_parquet("../data/parquet_format/train_data/compagnies.gzip")
df_vols = pd.read_parquet("../data/parquet_format/train_data/vols.gzip")
df_fuel = pd.read_parquet("../data/parquet_format/train_data/prix_fuel.gzip")
df_test = pd.read_parquet("../data/parquet_format/test_data/vols.gzip")

In [39]:
df_aeroports.head()

,CODE IATA,NOM,LIEU,PAYS,LONGITUDE,LATITUDE,HAUTEUR,PRIX RETARD PREMIERE 20 MINUTES,PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES
0,MCT,Muscat International Airport,Muscat,OM,58.284400939941406,23.593299865722656,48.0,53,3
1,SOU,Southampton Airport,Southampton,GB,-1.3567999601364136,50.95029830932617,44.0,24,5
2,PNH,Phnom Penh International Airport,Phnom Penh,KH,104.84400177001953,11.546600341796875,40.0,33,3
3,BLR,Kempegowda International Airport,Bangalore,IN,77.706299,13.1979,3000.0,70,9
4,FFD,RAF Fairford,Fairford,GB,-1.7900300025900002,51.6822013855,286.0,65,3


In [4]:
list_compagnies = df_compagnies['COMPAGNIE'].unique().tolist()
list_airports = df_aeroports['CODE IATA'].unique().tolist()

In [5]:
airlines = [random.choice(list_compagnies) for i in range(100)]
airports = [random.choice(list_airports) for i in range(100)]
nb_passagers = list(random.sample(range(0, 2505), 100))

In [6]:
y_preds = np.array(random.sample(range(-100, 1898), 100))
prediction = pd.DataFrame({'RETARD': y_preds, 'COMPAGNIES': airlines, 'AEROPORTS': airports,
                          'NOMBRE DE PASSAGERS': nb_passagers})

# METRICS 

On a choisit la **RMSE** : La racine de l'erreur quadratique moyenne ou racine de l'écart quadratique moyen est une mesure fréquemment utilisée des différences entre les valeurs prédites par un modèle ou estimateur et les valeurs observées

- **Formule** : $RMSE = \sqrt{MSE(\theta)} = \sqrt{\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)^{2}}$

- **Interpretation** : plus la valeur de notre RMSE est grande plus notre prédiction (le retard prédit à l'arrivée) est loin de la réalité terrain, du retard à l'arrivée effectif. Une RMSE proche de 0 signifie que notre modèle est proche de la réalité dans ces prédictions. 

Moyenne : $\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)$

# KPIs

In [7]:
prediction["CHIFFRE D'AFFAIRE COMPAGNIE"] = prediction['COMPAGNIES'].map(lambda x:\
                                       df_compagnies[df_compagnies['COMPAGNIE'] ==x]['CHIFFRE D AFFAIRE'].values[0])

In [41]:
prediction

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,CHIFFRE D'AFFAIRE COMPAGNIE
0,1449,Air Penguin,MWX,1958,31064000000
1,541,Fliying Is Possible Inc.,GLA,1288,2979000000
2,129,Air Piloter Sans Les Mains,DUB,928,7651000000
3,1286,IE 6.0 Flights,ERZ,1609,2660000000
4,1444,Ne Va Pas Partout Airlines,DEL,110,6235000000
...,...,...,...,...,...
95,1254,Air Penguin,WUH,681,31064000000
96,1134,Fliying Is Possible Inc.,LTN,1256,2979000000
97,1696,Overpriced Air,BWN,968,6649000000
98,263,Air Penguin,SCQ,2067,31064000000


In [9]:
prediction_avec_retard = prediction[prediction['RETARD']>0].copy()

### Prix du retard

Hypothese prix retard aéroport (centaine d'euros) : 
- après 10min :  la compagnie paye toutes les minutes le prix indiqué dans la colonne "PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES"
- après 20min : la compagnie paye un **supplément** qui est le prix indiqué dans la colonne "PRIX RETARD PREMIERE 20 MINUTES"

In [10]:
def cost_of_delay(pred_vol):
    delay = pred_vol['RETARD']
    compagny = pred_vol['COMPAGNIES']
    airport = pred_vol['AEROPORTS']

    twenty_first_min_cost = df_aeroports[df_aeroports['CODE IATA'] == airport]['PRIX RETARD PREMIERE 20 MINUTES'].values[0]
    ten_min_delay_cost = df_aeroports[
        df_aeroports['CODE IATA'] == airport]['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'].values[0]

    cost = 0
    if delay > 10 : 
        cost += ten_min_delay_cost * (delay - 10) 
    if delay >= 20 : 
        cost += twenty_first_min_cost
    return cost

In [11]:
prediction_avec_retard['COUT DU RETARD'] = prediction_avec_retard.apply(cost_of_delay, axis=1)

In [12]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD
0,1449,Air Penguin,MWX,1958,31064000000,6756
1,541,Fliying Is Possible Inc.,GLA,1288,2979000000,3786
2,129,Air Piloter Sans Les Mains,DUB,928,7651000000,2414
3,1286,IE 6.0 Flights,ERZ,1609,2660000000,12924
4,1444,Ne Va Pas Partout Airlines,DEL,110,6235000000,6896
...,...,...,...,...,...,...
95,1254,Air Penguin,WUH,681,31064000000,12716
96,1134,Fliying Is Possible Inc.,LTN,1256,2979000000,7804
97,1696,Overpriced Air,BWN,968,6649000000,4712
98,263,Air Penguin,SCQ,2067,31064000000,1619


### Indemnisation des clients 

Hypothèse : 
- 10% des clients vont demander à être indemnisé pour un retard compris entre 10min et 45min
    - Indemnité à payer : 1/4 du prix du billet
- 20% des clients vont demander à être indemnisé pour un retard supérieur à 1h 
    - Indemnité à payer : 1/2 du prix du billet
- 50% des clients vont demander à être indemnisé pour un retard supérieur à 3h 
    - Indemnité à payer : totalité du prix du billet
    
On fait l'hypothèse d'un fixe maximal du prix du billet : **200€**

In [13]:
def get_number_of_indemnities_asked(pred_vol): 
    delay = pred_vol.loc['RETARD']
    nb_of_passenger = pred_vol.loc['NOMBRE DE PASSAGERS']
    nb_of_indemnities_asked = 0
    if delay > 10 and delay <45: 
        nb_of_indemnities_asked = 20*nb_of_passenger//100
    elif delay > 60 and delay <180:
        nb_of_indemnities_asked = 50*nb_of_passenger//100
    elif delay > 180:
        nb_of_indemnities_asked = 75*nb_of_passenger//100
    return nb_of_indemnities_asked

def compensation_due(pred_vol, ticket_price=400): 
    delay = pred_vol.loc['RETARD']
    nb_of_indemnities_asked = pred_vol.loc["NOMBRE D'INDEMNITES DEMANDEES"]
    compensation_due_to_clients = 0
    if delay > 10 and delay <45: 
        compensation_due_to_clients = (ticket_price/3)*nb_of_indemnities_asked
    elif delay > 60 and delay <180:
        compensation_due_to_clients = (ticket_price/2)*nb_of_indemnities_asked
    elif delay > 180:
        compensation_due_to_clients = ticket_price*nb_of_indemnities_asked
    return compensation_due_to_clients

In [14]:
prediction_avec_retard[
    "NOMBRE D'INDEMNITES DEMANDEES"] = prediction_avec_retard.apply(get_number_of_indemnities_asked, axis=1)
prediction_avec_retard[
    "INDEMNITES A PAYER"] = prediction_avec_retard.apply(compensation_due, axis=1)

In [15]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,NOMBRE D'INDEMNITES DEMANDEES,INDEMNITES A PAYER
0,1449,Air Penguin,MWX,1958,31064000000,6756,1468,587200.0
1,541,Fliying Is Possible Inc.,GLA,1288,2979000000,3786,966,386400.0
2,129,Air Piloter Sans Les Mains,DUB,928,7651000000,2414,464,92800.0
3,1286,IE 6.0 Flights,ERZ,1609,2660000000,12924,1206,482400.0
4,1444,Ne Va Pas Partout Airlines,DEL,110,6235000000,6896,82,32800.0
...,...,...,...,...,...,...,...,...
95,1254,Air Penguin,WUH,681,31064000000,12716,510,204000.0
96,1134,Fliying Is Possible Inc.,LTN,1256,2979000000,7804,942,376800.0
97,1696,Overpriced Air,BWN,968,6649000000,4712,726,290400.0
98,263,Air Penguin,SCQ,2067,31064000000,1619,1550,620000.0


In [16]:
cost_of_delay_gb_airlines = prediction_avec_retard[["COMPAGNIES","CHIFFRE D'AFFAIRE COMPAGNIE", "COUT DU RETARD", "INDEMNITES A PAYER"]]\
                            .groupby(['COMPAGNIES'], as_index=False)\
                            .agg({
                                "CHIFFRE D'AFFAIRE COMPAGNIE":'first',
                                "COUT DU RETARD":'sum',
                                "INDEMNITES A PAYER":'sum'
                            })

In [17]:
cost_of_delay_gb_airlines["TOTAL A PAYER"] = cost_of_delay_gb_airlines["COUT DU RETARD"]\
                                            + cost_of_delay_gb_airlines["INDEMNITES A PAYER"]

In [22]:
cost_of_delay_gb_airlines["NV CHIFFRE D'AFFAIRE"] = cost_of_delay_gb_airlines["CHIFFRE D'AFFAIRE COMPAGNIE"]\
                                                    - cost_of_delay_gb_airlines["TOTAL A PAYER"]

# TODO : 

- Rajouter colonne avec pourcentage du chiffre d'affaire perdu 
-  Afficher nombre de vols en retard par compagnie 
- combien de clients ils vont perdre : plus de 3h de retard --> taux d'attrition de 5% des clients
- fréquence moyenne de réservation de vol avec la compagnie (3fs/an) : 5%x3x(prix du billet)
- afficher le détail répartition des couts dans total à payer dans les graphes

- NEXT STEPS 

In [40]:
cost_of_delay_gb_airlines

,COMPAGNIES,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,INDEMNITES A PAYER,TOTAL A PAYER,NV CHIFFRE D'AFFAIRE
0,Air Penguin,31064000000,54627,3183600.0,3238227.0,3.106076e+10
1,Air Piloter Sans Les Mains,7651000000,44553,2320400.0,2364953.0,7.648635e+09
2,Always A Problem Flights,51000000000,43390,2307200.0,2350590.0,5.099765e+10
3,Bel Air,3671000000,45374,2380600.0,2425974.0,3.668574e+09
4,Better Take A Train Airlines,5056000000,75136,3599800.0,3674936.0,5.052325e+09
5,Corporate Overlord Airways,40579000000,22639,2060000.0,2082639.0,4.057692e+10
6,Fliying Is Possible Inc.,2979000000,41201,2894000.0,2935201.0,2.976065e+09
7,IE 6.0 Flights,2660000000,49764,3973600.0,4023364.0,2.655977e+09
8,Morally Ambiguious Fligthts,6391000000,49338,2044400.0,2093738.0,6.388906e+09
9,Ne Va Pas Partout Airlines,6235000000,15593,1566000.0,1581593.0,6.233418e+09


In [29]:
fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "NV CHIFFRE D'AFFAIRE"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "TOTAL A PAYER"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

In [42]:
import plotly.graph_objects as go

for idx, company in enumerate(cost_of_delay_gb_airlines["COMPAGNIES"]):
    labels = ["NV CHIFFRE D'AFFAIRE","TOTAL A PAYER"]
    values = [cost_of_delay_gb_airlines.iloc[idx]["NV CHIFFRE D'AFFAIRE"],
              cost_of_delay_gb_airlines.iloc[idx]["TOTAL A PAYER"]]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
    #fig.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig.update_layout(title_text=company)
    fig.show()